# Performance Attribution #

In [1]:
# Import Libraries

# Data Management
import pandas as pd

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from data_downloader import get_market_data

In [2]:
# Now import the premiums
premiums_df = pd.read_csv(r'..\additional_data\famafrench_premiums.csv')
premiums_df.set_index('Date', inplace=True)
premiums_df.index = pd.to_datetime(premiums_df.index)
premiums_df.columns = ['mkt_premium', 'smb_premium', 'hml_premium', 'risk_free_rate']
premiums_df = premiums_df.div(100)

premiums_df

In [3]:
# And we can import the betas
ff_betas = pd.read_csv(r'..\additional_data\famafrench_betas.csv')
ff_betas.rename(columns={'Unnamed: 0': 'stocks'}, inplace=True)
ff_betas.set_index('stocks', inplace=True)

ff_betas

### Fama and French Attribution ###

In [4]:
# Import Stock Data
ticker = 'MSFT'

# Import Data
data_stock = get_market_data(
    ticker=ticker, 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

data_stock

In [5]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(data_stock['returns'].cumsum(), label=f'{ticker} Returns', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [6]:
# Now let us calculate the returns according to the beta
mkt_contribution = ff_betas.loc[ticker]['mkt_beta'] * premiums_df['mkt_premium']
mkt_contribution.name = 'mkt_contribution'

# For Size
size_contribution = ff_betas.loc[ticker]['smb_beta'] * premiums_df['smb_premium']
size_contribution.name = 'size_contribution'

# For Value
value_contribution = ff_betas.loc[ticker]['hml_beta'] * premiums_df['hml_premium']
value_contribution.name = 'value_contribution'

In [7]:
# And to calculate the residuals we just have to take the difference
residual_returns = data_stock['returns'] - (mkt_contribution + size_contribution + value_contribution)

In [8]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(data_stock['returns'].cumsum(), label=f'{ticker} Returns', alpha=1)
plt.plot(mkt_contribution.cumsum(), label='Market Factor', alpha=1)
plt.plot(size_contribution.cumsum(), label='Size Factor', alpha=1)
plt.plot(value_contribution.cumsum(), label='Value Factor', alpha=1)
plt.plot(residual_returns.cumsum(), label='Residual Returns', alpha=1)

# Config
plt.title('Attribution Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show() 

### Returns Contribution ###

In [9]:
# Calculate the mean contributions
mkt__mean_contribution = mkt_contribution.mean() 
size_mean_contribution = size_contribution.mean()
value_mean_contribution = value_contribution.mean()
total_mean_returns = data_stock['returns'].mean()
residual_mean_contribution = data_stock['returns'].mean() - mkt__mean_contribution - size_mean_contribution - value_mean_contribution

print(f'Market Mean Contribution: {mkt__mean_contribution * 25200}')
print(f'Size Mean Contribution: {size_mean_contribution * 25200}')
print(f'Value Mean Contribution: {value_mean_contribution * 25200}')
print(f'Residual Mean Contribution: {residual_mean_contribution * 25200}')

In [10]:
# Calculate percentages
pct_res = residual_mean_contribution / total_mean_returns * 100
pct_mkt = mkt__mean_contribution / total_mean_returns * 100
pct_smb = size_mean_contribution / total_mean_returns * 100
pct_hml = value_mean_contribution / total_mean_returns * 100

print("Returns Attribution (%)")
print(f"Residuals: {pct_res:.2f}%")
print(f"Market: {pct_mkt:.2f}%")
print(f"SMB: {pct_smb:.2f}%")
print(f"HML: {pct_hml:.2f}%")